In [ ]:
! pip install streamlit -q

In [ ]:
! pip install autogluon.tabular==1.1.1 -q

In [ ]:
!wget -q -O - ipv4.icanhazip.com

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%%writefile app.py
import streamlit as st
import librosa
import numpy as np
from sklearn.svm import SVC
from autogluon.tabular import TabularPredictor
import pandas as pd
import pickle as pkl
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
import pandas as pd


path_Autogluon = "/content/drive/My Drive/ProyectoTesis/Autogluon/Modelos2"
path_LSTM = "/content/drive/My Drive/ProyectoTesis/LSTM/LRModelHistories.pkl"
path_svm_kernels = "/content/drive/My Drive/ProyectoTesis/SVM/Streamlit/"
path_scaler = "/content/drive/My Drive/ProyectoTesis/Scaler.pkl"

with open(path_scaler, 'rb') as file:
  scaler = pkl.load(file)

# PATH="Models/Autogluon/"
# Dummy models (replace with pre-trained models or train new ones)
def predict(model_name, features):

  with open(path_scaler, 'rb') as file:
    scaler = pkl.load(file)

  if model_name == "SVM":

    models_svm = []
    import os
    for file_name in os.listdir(path_svm_kernels):
      path = os.path.join(path_svm_kernels, file_name)
      with open(path, 'rb') as file:
        models_svm.append((file_name[:-4], pkl.load(file)))

    features, columns = features
    data = [[np.mean(features[i]) for i in range(len(features))]]
    data = scaler.transform(data)

    results = {}

    for name, mod in models_svm:
      results[name] = "Bonafide" if mod.predict(data)[0] == 0 else "Spoof"

    st.write(results)

    pass
  elif model_name == "Autogluon":
    features, columns = features
    data = [[np.mean(features[i]) for i in range(len(features))]]
    data = scaler.transform(data)
    data = pd.DataFrame(data, columns=columns)

    predictor = TabularPredictor.load(path_Autogluon)
    autogluon_models = predictor.model_names()[:-1]
    results = {}

    for model in autogluon_models:
      prediction = predictor.predict(data, model=model)[0]
      prediction = "Bonafide" if prediction == 0 else "Spoof"
      results[model] = prediction

    st.write(results)


  #LSTM
  else:
    with open(path_LSTM, 'rb') as file:
      models = pkl.load(file)

    features, columns = features
    st.write(len(features))
    features = np.array(features)
    new_features = np.transpose(features, (1,0))

    scaled_features = np.array([scaler.transform(new_features)])
    new_features = np.array([new_features])
    #st.write(scaled_features)
    results = {}

    for mod in models:
      hist, mod, lr, ttime = mod
      pred = mod.predict(new_features)
      results[lr] = pred
    st.write(results)

def specificModel():
  predictor = TabularPredictor.load(path_Autogluon)
  autogluon_models = predictor.model_names()[:-1]
  svm_models = ['SVM_POLYNOMIAL', 'SVM_RBF', 'SVM_SIGMOID']
  svm_models.extend([f'SVM_RADIAL_C_{10**i}' for i in range(-1, 3)])
  lrs = [0.0001, 0.0005, 0.001, 0.005, 0.01]
  lstm_models = [f"LSTM_LR_{lrs[i]}" for i in range(len(lrs))]
  models = autogluon_models + svm_models + lstm_models

# Feature extraction
def extract_features(audio):
    y, sr =  librosa.load(audio)
    sr = 16000
    features = {
        'zcr': librosa.feature.zero_crossing_rate(y)[0],
        'rmse': librosa.feature.rms(y=y)[0],
        'spectral_centroid': librosa.feature.spectral_centroid(y=y, sr=sr)[0],
        'spectral_bandwidth': librosa.feature.spectral_bandwidth(y=y, sr=sr)[0],
        'spectral_rolloff': librosa.feature.spectral_rolloff(y=y, sr=sr)[0],
        'spectral_contrast': librosa.feature.spectral_contrast(y=y, sr=sr)[0],
        'chroma_stft': librosa.feature.chroma_stft(y=y, sr=sr)[0],
    }
    # Add all 13 MFCCs
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=10)

    for i in range(10):
        features[f'mfcc{i+1}'] = mfccs[i]

    ret_value = [val for val in features.values()]
    return ret_value, features.keys()

# App
st.title("Audio Feature Extraction and Prediction")

# Upload audio file
uploaded_file = st.file_uploader("Upload an audio file", type=["wav", "mp3", "flac"])

if uploaded_file is not None:
    # Save uploaded file temporarily
    with open("temp_audio_file.wav", "wb") as f:
        f.write(uploaded_file.read())

    # Extract features
    st.write("Extracting features...")
    features = extract_features("temp_audio_file.wav")
    st.write(f"Extracted features: {len(features[0])}")

    # Model selection
    st.write("Select a Machine Learning Model:")
    models = ['SVM', 'Autogluon', 'LSTM']
    model_name = st.selectbox("Choose a model", models)

    # Train model with dummy data (replace this with actual training and models)
    if st.button("Predict"):
      predict(model_name, features)


In [ ]:
!streamlit run app.py & npx localtunnel --port 8501